<a href="https://colab.research.google.com/github/tilaboy/nlp_transformer_tutorial/blob/main/learning_notes/ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import get_dataset_config_names 

In [ ]:
xtreme_subsets = get_dataset_config_names("xtreme") 
print(f"XTREME has {len(xtreme_subsets)} configurations") 
